In [1]:
import dotenv
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_openai import AzureOpenAIEmbeddings
dotenv.load_dotenv()
from langchain_community.vectorstores import Chroma
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_similarity,
    answer_correctness,
)
from datasets import Dataset
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
import os
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
import getpass
from langchain.retrievers.document_compressors import CohereRerank
from langchain.retrievers import ContextualCompressionRetriever
import sys
sys.tracebacklimit = 0

c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL")
OPENAI_DEPLOYMENT = os.environ.get("OPENAI_DEPLOYMENT")
EMBEDDING_MODEL = os.environ.get("EMBEDDING_MODEL")
EMBEDDING_DEPLOYMENT = os.environ.get("EMBEDDING_DEPLOYMENT")
OPENAI_MODEL_GPT4 = os.environ.get("OPENAI_MODEL_GPT4")
OPENAI_DEPLOYMENT_GPT4 = os.environ.get("OPENAI_DEPLOYMENT_GPT4")

In [3]:
embeddings_client = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDING_DEPLOYMENT,
    openai_api_version=OPENAI_API_VERSION)
llm = AzureChatOpenAI(model_name=OPENAI_MODEL, azure_deployment=OPENAI_DEPLOYMENT,temperature=0)
llm_gpt4 = AzureChatOpenAI(model_name=OPENAI_MODEL_GPT4, azure_deployment=OPENAI_DEPLOYMENT_GPT4,temperature=0)

In [4]:
def evaluation_llm(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
    )
    return result

In [5]:
def evaluation_rag(questions, answers, contexts, ground_truths):
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truths": ground_truths
    }
    dataset = Dataset.from_dict(data)
    azure_configs = {
        "base_url": AZURE_OPENAI_ENDPOINT,
        "model_deployment": OPENAI_DEPLOYMENT,
        "model_name": OPENAI_MODEL,
        "embedding_deployment": EMBEDDING_DEPLOYMENT,
        "embedding_name": EMBEDDING_MODEL,  # most likely
    }

    azure_model = AzureChatOpenAI(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["model_deployment"],
        model=azure_configs["model_name"],
        validate_base_url=False,
    )

    azure_embeddings = AzureOpenAIEmbeddings(
        openai_api_version=OPENAI_API_VERSION,
        azure_endpoint=azure_configs["base_url"],
        azure_deployment=azure_configs["embedding_deployment"],
        model=azure_configs["embedding_name"],
    )
    result = evaluate(
        dataset = dataset, 
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall,
            answer_similarity,
            answer_correctness,
        ], 
        llm=azure_model, 
        embeddings=azure_embeddings,
        raise_exceptions=False,
    )
    return result

In [6]:
questions = [
    "What happened to Alexei Nivalny in February 2024?",
    "What were the initial findings of the National Transportation Safety Board (NTSB) regarding the door blowout incident on an Alaska Airlines Boeing 737 Max 9, and how did Boeing respond to the report?",
    "What is the current financial situation in Europe regarding support for Ukraine, and what potential challenges might arise in the next 12 months?",
    "What is Prince William's stance on the Israel-Gaza conflict, and what key points did he emphasize during his statement?",
    "What are the key details regarding the arrest of Daniela Klette, an alleged member of the Red Army Faction (RAF), after more than 30 years in hiding?",
    "How does the entertainment industry, particularly Hollywood, respond to the introduction of OpenAI's Sora text-to-video generator, and what concerns and actions have been expressed by industry professionals?",
    "What are the key reasons for the conflict between Israel and Hamas in Gaza?",
    "What are some key factors influencing the upcoming 2024 US presidential election, especially in terms of the candidates' strategies, key battleground states, and significant issues?",
    "How has Boeing responded to recent criticisms and safety concerns, and what steps has the company taken to address the identified issues in its safety culture and aircraft manufacturing processes?",
    "What recent actions and statements by Kim Jong Un have raised concerns about the possibility of North Korea engaging in military conflict, and what factors do experts highlight as potential obstacles to such a war?"
    ]

ground_truths = [
    ["Alexei Navalny died in his prison cell in Siberia on February 16. He was serving a 19-year sentence on charges widely seen as politically motivated. Navalny's colleague, Maria Pevchikh, claimed that he was about to be freed in a prisoner swap. The proposed exchange involved Navalny being exchanged for Vadim Krasikov, a Russian hitman serving a life sentence for murder in Germany. Two US citizens held in Russia were also reportedly part of the deal. However, Kremlin spokesperson Dmitry Peskov denied knowledge of such agreements. Maria Pevchikh stated that negotiations for the prisoner swap had been ongoing for two years and were in their final stage on February 15. Navalny's death occurred on February 16, with prison officials attributing it to him falling ill after a walk. Pevchikh claimed that Putin changed his mind about the deal at the last minute, and the Russian president 'could not tolerate Navalny being free.' The reasons behind Putin's potential agreement to the swap and subsequent change of mind remain unclear. Navalny's widow, Yulia, has accused Putin of killing her husband, and Western leaders have blamed Putin for Navalny's death. Investigations into Navalny's death were ongoing at the time of the provided text, and there were discussions of potential sanctions by the EU and the US on Russia following Navalny's death."],
    ["The National Transportation Safety Board (NTSB) conducted an investigation into the door blowout incident on an Alaska Airlines Boeing 737 Max 9. The initial findings revealed that four key bolts meant to secure the unused door to the fuselage were missing, leading to the door's detachment shortly after takeoff. Boeing accepted accountability for the incident, with its president, Dave Calhoun, acknowledging the need for improvement. Boeing outlined a comprehensive plan to strengthen quality and regain stakeholders' confidence, emphasizing increased inspections, documentation of door plug assembly, and additional scrutiny in the supply chain. Calhoun assured a commitment to transparency and collaboration with regulatory authorities in addressing safety concerns."],
    ["Europe is grappling with the question of how long it can sustain draining financial support for Ukraine, having spent over $100 billion since the crisis began. The war is deadlocked, and recent setbacks for Ukraine, coupled with delays in US financial aid approval, are causing concerns. As fatigue sets in and political pressure grows, there's uncertainty about continued funding, especially if the US pulls financial support. European officials are considering using frozen Russian assets for funding, but concerns about setting precedents and challenges in independently providing weapons make the situation complex. The next 12 months are crucial, with European elections approaching, and maintaining support for Ukraine is deemed essential, even if the US reduces its involvement."],
    ["Prince William expressed deep concern about the human cost of the conflict since the Hamas terrorist attack, called for an end to the fighting, emphasized the need for increased humanitarian support to Gaza, and urged the release of hostages. He highlighted the critical importance of aid delivery and acknowledged the severe humanitarian situation in the region. Additionally, Prince William emphasized the impact on civilians and expressed hope for a brighter future and permanent peace."],
    ["Daniela Klette, a 65-year-old alleged RAF member from the so-called third generation, was arrested in the Berlin district of Kreuzberg after three decades in hiding. She is accused of attempted murder and serious robberies, with charges dating back to the 1980s and 1990s. Klette was identified through fingerprints, did not resist arrest, and has been flown to Bremen for pre-trial detention. A second person, with unconfirmed identity, was also arrested. The arrest is considered a milestone in the fight against 'tterrorism,' with officials highlighting the persistence of pursuing criminals regardless of the time elapsed since their crimes."],
    ["The entertainment industry, including Hollywood, is cautiously reacting to OpenAI's Sora text-to-video generator. While some acknowledge the potential impact on job roles, others express concerns about the tool's current limitations, such as temporal consistency issues and artifacts. Filmmakers like Tyler Perry have taken action, delaying studio expansions due to AI advancements. Industry professionals are advocating for protections around the use of AI models, considering potential job redundancies. OpenAI, aware of these concerns, has expanded licensing agreements and expressed intentions to engage with policymakers, educators, and artists before making Sora publicly available. Concerns also include potential copyright issues and the generation of actors' likenesses without consent. The industry remains hopeful that it can adapt to AI innovations, emphasizing the irreplaceable role of the human creative mind."],
    ["The conflict between Israel and Hamas in Gaza stems from Hamas' desire to create an Islamic state, rejection of Israel's right to exist, and its justification for attacks based on perceived Israeli crimes against Palestinians. The recent escalation involves a series of Hamas attacks on Israel, leading to a significant military response and the ongoing hostage situation. The conflict has resulted in a high death toll, displacement of civilians, and severe humanitarian crises in Gaza."],
    ["The 2024 US presidential election is characterized by a unique rematch between Joe Biden and Donald Trump. Both candidates face challenges and vulnerabilities. Trump won the South Carolina Republican primary, strengthening his path to the GOP nomination. The election is expected to be decided in a few key states, such as Wisconsin, Pennsylvania, Michigan, Arizona, and Georgia. Economic factors, including positive indicators but lingering public concerns, play a role. The campaigns are likely to focus on issues like abortion and immigration, with Democrats emphasizing Trump's impact on abortion laws and Republicans highlighting immigration concerns. Other potential factors include crime rates, the environment, climate change, and foreign policy. The candidates' age, health, and the possibility of third-party candidates add uncertainty to the election landscape. Trump's legal challenges and the aftermath of the January 2021 Capitol attack could also influence public opinion."],
    ["Boeing has responded to recent criticisms and safety concerns by acknowledging accountability for incidents, such as the door blowout on an Alaska Airlines Boeing 737 Max 9. The company's president, Dave Calhoun, emphasized transparency and outlined a comprehensive plan to strengthen quality, including increased inspections, documentation of door plug assembly, and additional scrutiny in the supply chain. Boeing has also undergone organizational changes, with the departure of the head of the 737 program and the appointment of new leadership to enhance focus on safety and quality. Additionally, a recent FAA report highlighted major issues with Boeing's safety culture, and Boeing expressed support for the panel's review, acknowledging the need for continuous improvement in fostering a safety culture within the company."],
    ["Recent actions and statements by Kim Jong Un, including renouncing the goal of peaceful reunification with South Korea, intensifying nuclear threats, and mentioning South Korea as the 'principal enemy, have raised concerns about the potential for North Korea to initiate a military conflict. Experts point out factors such as the lack of support from China and Russia, North Korea's inferior conventional weapons, and doubts about Kim's willingness to risk his regime's stability as obstacles to a full-scale war. Despite increased tensions, some experts believe that provocations may be aimed at negotiation strategies or domestic stability rather than an imminent military attack."]
    ]

### General answer by LLMs

In [7]:
template = """{question}"""
prompt = ChatPromptTemplate.from_template(template)

In [8]:
llm_chain =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm}
)
llm_chain_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt | llm_gpt4}
)

In [9]:
answers_llm = []
contexts_llm = [[""],[""],[""],[""],[""],[""],[""],[""],[""],[""]]

In [10]:
for query in questions:
    response = llm_chain.invoke({"question": query})
    answers_llm.append(response["response"].content)

In [11]:
llm_results = evaluation_llm(questions, answers_llm, contexts_llm, ground_truths)
print(llm_results)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 40/40 [00:10<00:00,  3.89it/s]


{'faithfulness': 0.9630, 'answer_relevancy': 0.4663, 'answer_similarity': 0.9008, 'answer_correctness': 0.4650}


In [13]:
answers_llm_gpt4 = []
for query in questions:
    response = llm_chain_gpt4.invoke({"question": query})
    answers_llm_gpt4.append(response["response"].content)
llm_results_gpt4 = evaluation_llm(questions, answers_llm_gpt4, contexts_llm, ground_truths)
print(llm_results_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 40/40 [00:14<00:00,  2.71it/s]


{'faithfulness': 0.9167, 'answer_relevancy': 0.2835, 'answer_similarity': 0.9067, 'answer_correctness': 0.5423}


In [31]:
def dictionary(result):
    dict_result = dict(result)
    average_score = sum(dict_result.values()) / len(dict_result)
    print(f"The average score is: {average_score}")
    return average_score

# RAG

### Naive RAG

In [13]:
def retrieval_chain(prompt, retriever, llm):
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
    )
    return retrieval_augmented_qa_chain

In [25]:
loader = DirectoryLoader('../news', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db_naive = Chroma.from_documents(chunks, embeddings_client, persist_directory = "../news/vectordb/naive")
db_naive.persist()
retriever_naive = db_naive.as_retriever()

In [19]:
db_naive = Chroma(persist_directory = "../news/vectordb/naive", embedding_function=embeddings_client)
retriever_naive = db_naive.as_retriever()

In [7]:
template = """{question}. 
Provided context {context}."""
prompt = ChatPromptTemplate.from_template(template)

In [21]:
answers_naive = []
contexts_naive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_naive, llm).invoke({"question": query})
        # Access the response content
        answers_naive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_naive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_naive.append("No answer")
        context_full = retriever_naive.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_naive.append(context_content)

In [22]:
result_naive_rag = evaluation_rag(questions, answers_naive, contexts_naive, ground_truths)
print(result_naive_rag)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.94it/s]


{'faithfulness': 0.9187, 'answer_relevancy': 0.8303, 'context_precision': 0.9667, 'context_recall': 0.9600, 'answer_similarity': 0.9630, 'answer_correctness': 0.7468}


### recursive splitter

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder()
chunks_r = text_splitter.split_documents(documents)
db_basic = Chroma.from_documents(chunks_r, embeddings_client, persist_directory = "../news/vectordb/recursive_basic")
db_basic.persist()
retriever_basic = db_basic.as_retriever()

In [23]:
db_basic = Chroma(persist_directory = "../news/vectordb/recursive_basic", embedding_function=embeddings_client)
retriever_basic = db_basic.as_retriever()

In [24]:
answers_recursive = []
contexts_recursive = []
for query in questions:
    try:  
        response = retrieval_chain(prompt, retriever_basic, llm).invoke({"question": query})
        # Access the response content
        answers_recursive.append(response["response"].content)
        # Access the context content
        context_content = [context.page_content for context in response["context"]]
        contexts_recursive.append(context_content)  
    except Exception as e:  
        print(f"Warning: {e}" + "on the following question: " + query)  
        answers_recursive.append("No answer")
        context_full = retriever_basic.get_relevant_documents(query)
        context_content = [context.page_content for context in context_full]
        contexts_recursive.append(context_content)

In [25]:
result_recursive = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
print(result_recursive)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.76it/s]


{'faithfulness': 0.9524, 'answer_relevancy': 0.8284, 'context_precision': 0.9750, 'context_recall': 0.9800, 'answer_similarity': 0.9634, 'answer_correctness': 0.7330}


### chunk sizes

In [11]:
def run_and_evaluate(retriever, prompt, llm):
    answers_recursive = []
    contexts_recursive = []

    for query in questions:
        try:  
            response = retrieval_chain(prompt, retriever, llm).invoke({"question": query})
            # Access the response content
            answers_recursive.append(response["response"].content)
            # Access the context content
            context_content = [context.page_content for context in response["context"]]
            contexts_recursive.append(context_content)  
        except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_recursive.append("No answer")
            context_full = retriever.get_relevant_documents(query)
            context_content = [context.page_content for context in context_full]
            contexts_recursive.append(context_content)


    result = evaluation_rag(questions, answers_recursive, contexts_recursive, ground_truths)
    return result

chunk = 1000

In [ ]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 1000, chunk_overlap = 100)
chunks_1000 = text_splitter.split_documents(documents)
print(len(chunks_1000))
db_1000 = Chroma.from_documents(chunks_1000, embeddings_client, persist_directory = "../news/vectordb/recursive_1000")
db_1000.persist()

In [27]:
db_1000 = Chroma(persist_directory = "../news/vectordb/recursive_1000", embedding_function=embeddings_client)

In [28]:
retriever_1000 = db_1000.as_retriever()
result_1000 = run_and_evaluate(retriever_1000, prompt, llm)
print("CHUNK SIZE 1000")
print(result_1000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:14<00:00,  4.18it/s]


CHUNK SIZE 1000
{'faithfulness': 0.9841, 'answer_relevancy': 0.8044, 'context_precision': 0.9750, 'context_recall': 0.9550, 'answer_similarity': 0.9614, 'answer_correctness': 0.6946}


chunk = 500

In [33]:
# THE FIRST TIME RUN THIS, AFTER RUN THE NEXT CELL
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 500, chunk_overlap = 50)
chunks_500 = text_splitter.split_documents(documents)
print(len(chunks_500))
db_500 = Chroma.from_documents(chunks_500, embeddings_client, persist_directory = "../news/vectordb/recursive_500")
db_500.persist()

107


In [ ]:
db_500 = Chroma(persist_directory = "../news/vectordb/recursive_500", embedding_function=embeddings_client)

In [34]:
retriever_500 = db_500.as_retriever()
result_500 = run_and_evaluate(retriever_500, prompt, llm)
print("CHUNK SIZE 500")
print(result_500)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.89it/s]


CHUNK SIZE 500
{'faithfulness': 0.9378, 'answer_relevancy': 0.7268, 'context_precision': 0.9500, 'context_recall': 0.9472, 'answer_similarity': 0.9616, 'answer_correctness': 0.7098}


chunk = 2000

In [35]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks_2000 = text_splitter.split_documents(documents)
print(len(chunks_2000))
db_2000 = Chroma.from_documents(chunks_2000, embeddings_client, persist_directory = "../news/vectordb/recursive_2000")


37


In [8]:
db_2000 = Chroma(persist_directory = "../news/vectordb/recursive_2000", embedding_function=embeddings_client)

In [37]:
retriever_2000 = db_2000.as_retriever()
result_2000 = run_and_evaluate(retriever_2000, prompt, llm)
print("CHUNK SIZE 2000")
print(result_2000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:14<00:00,  4.28it/s]


CHUNK SIZE 2000
{'faithfulness': 0.9435, 'answer_relevancy': 0.8731, 'context_precision': 0.9750, 'context_recall': 1.0000, 'answer_similarity': 0.9636, 'answer_correctness': 0.7390}


chunk = 3000

In [38]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 3000, chunk_overlap = 300)
chunks_3000 = text_splitter.split_documents(documents)
print(len(chunks_3000))
db_3000 = Chroma.from_documents(chunks_3000, embeddings_client, persist_directory = "../news/vectordb/recursive_3000")

32


In [ ]:
db_3000 = Chroma(persist_directory = "../news/vectordb/recursive_3000", embedding_function=embeddings_client)

In [39]:
retriever_3000 = db_3000.as_retriever()
result_3000 = run_and_evaluate(retriever_3000, prompt, llm)
print("CHUNK SIZE 3000")
print(result_3000)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:13<00:00,  4.30it/s]


CHUNK SIZE 3000
{'faithfulness': 0.9592, 'answer_relevancy': 0.7520, 'context_precision': 0.9583, 'context_recall': 0.9800, 'answer_similarity': 0.9645, 'answer_correctness': 0.7113}


### now time to look for different top-k

Note: We continue with the size chunk of 2000 as it had the highest average score

In [9]:
retriever_2000 = db_2000.as_retriever()

In [14]:
retriever_2 = db_2000.as_retriever(search_kwargs={"k": 2})
result_2 = run_and_evaluate(retriever_2, prompt, llm)
print("CHUNK SIZE 1000, K=2")
print(result_2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.54it/s]


CHUNK SIZE 1000, K=2
{'faithfulness': 0.9306, 'answer_relevancy': 0.9134, 'context_precision': 1.0000, 'context_recall': 0.9289, 'answer_similarity': 0.9617, 'answer_correctness': 0.7467}


In [15]:
retriever_3 = db_2000.as_retriever(search_kwargs={"k": 3})
result_3 = run_and_evaluate(retriever_3, prompt, llm)
print("CHUNK SIZE 1000, K=3")
print(result_3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:09<00:00,  6.10it/s]


CHUNK SIZE 1000, K=3
{'faithfulness': 0.9260, 'answer_relevancy': 0.8950, 'context_precision': 0.9833, 'context_recall': 0.9750, 'answer_similarity': 0.9641, 'answer_correctness': 0.7973}


In [16]:
retriever_5 = db_2000.as_retriever(search_kwargs={"k": 5})
result_5 = run_and_evaluate(retriever_5, prompt, llm)
print("CHUNK SIZE 1000, K=5")
print(result_5)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.76it/s]


CHUNK SIZE 1000, K=5
{'faithfulness': 0.9688, 'answer_relevancy': 0.7442, 'context_precision': 0.9617, 'context_recall': 0.9714, 'answer_similarity': 0.9619, 'answer_correctness': 0.7390}


In [17]:
retriever_6= db_2000.as_retriever(search_kwargs={"k": 6})
result_6 = run_and_evaluate(retriever_6, prompt, llm)
print("CHUNK SIZE 1000, K=5")
print(result_6)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:  97%|█████████▋| 58/60 [00:13<00:01,  1.38it/s]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 58, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "C:\Users\sigitalapina\AppData\Local\Programs\Python\Python311\Lib\asyncio\tasks.py", line 605, in _wait_for_one
    return f.result()  # May raise f.exception().
           ^^^^^^^^^^
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\ragas\executor.py", line 91, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sigitalapin

CHUNK SIZE 1000, K=5
{'faithfulness': 0.7835, 'answer_relevancy': 0.8474, 'context_precision': 0.8161, 'context_recall': 0.9210, 'answer_similarity': 0.9402, 'answer_correctness': 0.8747}


In [23]:
dict_res6 = dict(result_6)
average_score = sum(dict_res6.values()) / len(dict_res6)
print(f"The average score is: {average_score}")

The average score is: 0.8638028044818998


### look for different retrievers

3 chunks was the best score

#### parent document retriever

In [26]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

parent_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000)
child_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=200, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents",persist_directory = "../news/vectordb/parent", embedding_function=embeddings_client)
vectorstore.persist()
store = InMemoryStore()
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
parent_document_retriever.add_documents(documents)
result_parent = run_and_evaluate(parent_document_retriever, prompt, llm)
print(result_parent)
dict_result_parent = dict(result_parent)
average_score_parent = sum(dict_result_parent.values()) / len(dict_result_parent)
print(f"The average score is: {average_score_parent}")

TypeError: run_and_evaluate() missing 2 required positional arguments: 'prompt' and 'llm'

In [27]:
result_parent = run_and_evaluate(parent_document_retriever, prompt, llm)
print(result_parent)
dict_result_parent = dict(result_parent)
average_score_parent = sum(dict_result_parent.values()) / len(dict_result_parent)
print(f"The average score is: {average_score_parent}")

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.99it/s]


{'faithfulness': 0.9738, 'answer_relevancy': 0.9057, 'context_precision': 0.9833, 'context_recall': 0.9489, 'answer_similarity': 0.9621, 'answer_correctness': 0.7362}
The average score is: 0.9183376088705733


In [28]:
parent_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap = 50)
child_splitter_small = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_small",persist_directory = "../news/vectordb/parent_small", embedding_function=embeddings_client)
vectorstore.persist()
store = InMemoryStore()
parent_document_retriever_small = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_small,
    parent_splitter=parent_splitter_small,
)
parent_document_retriever_small.add_documents(documents)
result_parent_small = run_and_evaluate(parent_document_retriever_small, prompt, llm)
print(result_parent_small)
dict_result_parent_small = dict(result_parent_small)
average_score_parent_small = sum(dict_result_parent_small.values()) / len(dict_result_parent_small)
print(f"The average score is: {average_score_parent_small}")

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.75it/s]


{'faithfulness': 0.9844, 'answer_relevancy': 0.7338, 'context_precision': 0.9417, 'context_recall': 0.9378, 'answer_similarity': 0.9615, 'answer_correctness': 0.6669}
The average score is: 0.871012803992867


In [29]:
parent_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=3000)
child_splitter_large = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=300, chunk_overlap = 0)

vectorstore = Chroma(collection_name="split_parents_large",persist_directory = "../news/vectordb/parent_large", embedding_function=embeddings_client)
vectorstore.persist()
store = InMemoryStore()
parent_document_retriever_large = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter_large,
    parent_splitter=parent_splitter_large,
)
parent_document_retriever_large.add_documents(documents)
result_parent_large = run_and_evaluate(parent_document_retriever_large, prompt, llm)
print(result_parent_large)
dict_result_parent_large = dict(result_parent_large)
average_score_parent_large = sum(dict_result_parent_large.values()) / len(dict_result_parent_large)
print(f"The average score is: {average_score_parent_large}")

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:11<00:00,  5.10it/s]


{'faithfulness': 0.9738, 'answer_relevancy': 0.9087, 'context_precision': 0.9833, 'context_recall': 0.9639, 'answer_similarity': 0.9642, 'answer_correctness': 0.6708}
The average score is: 0.9107785434064702


#### Maximum marginal relevance retrieval

In [32]:
retriever_mmr = db_2000.as_retriever(search_type="mmr",search_kwargs={"k": 3})
result_mmr = run_and_evaluate(retriever_mmr, prompt, llm)
print("Marginal relevance")
print(result_mmr)
average_score_mmr = dictionary(result_mmr)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:10<00:00,  5.59it/s]


Marginal relevance
{'faithfulness': 0.9643, 'answer_relevancy': 0.7875, 'context_precision': 1.0000, 'context_recall': 0.9078, 'answer_similarity': 0.9574, 'answer_correctness': 0.6069}
The average score is: 0.8706369232635726


#### BM25

In [34]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 2000, chunk_overlap = 200)
chunks_2000 = text_splitter.split_documents(documents)

In [35]:
retriever_bm25 = BM25Retriever.from_documents(chunks_2000)
result_bm25 = run_and_evaluate(retriever_bm25, prompt, llm)
print("BM25")
print(result_bm25)
average_score_bm25 = dictionary(result_bm25)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]Task exception was never retrieved
future: <Task finished name='Task-625' coro=<AsyncClient.aclose() done, defined at c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py:2011> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_client.py", line 2018, in aclose
    await self._transport.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packages\httpx\_transports\default.py", line 385, in aclose
    await self._pool.aclose()
  File "c:\Users\sigitalapina\OneDrive - KPMG\Desktop\thesis-rag\.venv\Lib\site-packa

BM25
{'faithfulness': 0.9708, 'answer_relevancy': 0.7770, 'context_precision': 0.9333, 'context_recall': 0.9750, 'answer_similarity': 0.9483, 'answer_correctness': 0.6228}
The average score is: 0.8712076560157209


#### Ensambler - Hybrid

In [38]:
ensemble_retriever_1 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.75, 0.25])
result_ensemble1 = run_and_evaluate(ensemble_retriever_1, prompt, llm)
print("Ensambler 75/25")
print(result_ensemble1)
average_score_ensambler1 = dictionary(result_ensemble1)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:18<00:00,  3.23it/s]


Ensambler
{'faithfulness': 0.9607, 'answer_relevancy': 0.8681, 'context_precision': 0.9617, 'context_recall': 0.9500, 'answer_similarity': 0.9604, 'answer_correctness': 0.6588}
The average score is: 0.8932842741994689


In [39]:
ensemble_retriever_2 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.5, 0.5])
result_ensemble2 = run_and_evaluate(ensemble_retriever_2, prompt, llm)
print("Ensambler 50/50")
print(result_ensemble2)
avg_result_ensemble2 = dictionary(result_ensemble2)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:15<00:00,  3.84it/s]


Ensambler 50/50
{'faithfulness': 0.9841, 'answer_relevancy': 0.8430, 'context_precision': 0.9667, 'context_recall': 0.9500, 'answer_similarity': 0.9532, 'answer_correctness': 0.7541}
The average score is: 0.9085234951728957


In [40]:
ensemble_retriever_3 = EnsembleRetriever(retrievers=[retriever_bm25, retriever_3], weights=[0.25,0.75])
result_ensemble3 = run_and_evaluate(ensemble_retriever_3, prompt, llm)
print("Ensambler 25/75")
print(result_ensemble3)
avg_result_ensemble3 = dictionary(result_ensemble3)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:15<00:00,  3.90it/s]


Ensambler 25/75
{'faithfulness': 1.0000, 'answer_relevancy': 0.7409, 'context_precision': 0.9583, 'context_recall': 0.9214, 'answer_similarity': 0.9612, 'answer_correctness': 0.7724}
The average score is: 0.8923824405244444


#### Multi-stage - reranker

In [42]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [45]:
retriever_context = retriever_2000
compressor = CohereRerank(top_n = 3)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever_context
)

result_compression = run_and_evaluate(compression_retriever, prompt, llm)
print("Reranker")
print(result_compression)
avg_result_compression = dictionary(result_compression)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:15<00:00,  3.99it/s]


Reranker
{'faithfulness': 0.8968, 'answer_relevancy': 0.8880, 'context_precision': 0.9417, 'context_recall': 0.9750, 'answer_similarity': 0.9576, 'answer_correctness': 0.6623}
The average score is: 0.8868849276357208


#### creating context by remaking the query

In [46]:
template_context = "Generate a search query to fetch the relevant documents using the user's {question}. Craft a query that specifically targets the keywords in the question. In the answer provide only the query."
prompt_context = ChatPromptTemplate.from_template(template_context)

In [49]:
answers_final = []
contexts_final = []
llm_for_context =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm}
)
for query in questions:
    response_check = llm_for_context.invoke({"question": query})
    search_query = response_check["response"].content
    retrieval_augmented_qa_chain = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm, "context": itemgetter("context")}
)
    docs = retriever_3.get_relevant_documents(search_query)
    formatted_docs = []
    for doc in docs:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final.append(response["response"].content)
            contexts_final.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final.append("No answer")
            contexts_final.append(formatted_docs)


result_search_query = evaluation_rag(questions, answers_final, contexts_final, ground_truths)
print(result_search_query)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:12<00:00,  4.95it/s]


{'faithfulness': 0.9815, 'answer_relevancy': 0.8995, 'context_precision': 0.9833, 'context_recall': 0.9750, 'answer_similarity': 0.9605, 'answer_correctness': 0.7531}


In [50]:
print(result_search_query)
avg_result_search_query = dictionary(result_search_query)

{'faithfulness': 0.9815, 'answer_relevancy': 0.8995, 'context_precision': 0.9833, 'context_recall': 0.9750, 'answer_similarity': 0.9605, 'answer_correctness': 0.7531}
The average score is: 0.9254703293008256


### change model to GPT-4

In [53]:
answers_final_gpt4 = []
contexts_final_gpt4 = []
llm_for_context_gpt4 =(
    { "question": itemgetter("question")}
    | RunnablePassthrough()
    | {"response": prompt_context | llm_gpt4}
)
for query in questions:
    response_check_gpt4 = llm_for_context_gpt4.invoke({"question": query})
    search_query_gpt4 = response_check_gpt4["response"].content
    retrieval_augmented_qa_chain_gpt4 = (
        {"context": itemgetter("context"), "question": itemgetter("question")}
        | RunnablePassthrough.assign(context=itemgetter("context"))
        | {"response": prompt | llm_gpt4, "context": itemgetter("context")}
)
    docs_gpt4 = retriever_3.get_relevant_documents(search_query_gpt4)
    formatted_docs = []
    for doc in docs_gpt4:
        resulting_doc = doc.page_content
        formatted_docs.append(resulting_doc)
    try:  
            response = retrieval_augmented_qa_chain_gpt4.invoke({"context": formatted_docs, "question": query})
            # Access the response content
            answers_final_gpt4.append(response["response"].content)
            contexts_final_gpt4.append(formatted_docs)  
    except Exception as e:  
            print(f"Warning: {e}" + "on the following question: " + query)  
            answers_final_gpt4.append("No answer")
            contexts_final_gpt4.append(formatted_docs)


result_search_query_gpt4 = evaluation_rag(questions, answers_final_gpt4, contexts_final_gpt4, ground_truths)
print(result_search_query_gpt4)

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`
Evaluating: 100%|██████████| 60/60 [00:15<00:00,  3.95it/s]


{'faithfulness': 0.9907, 'answer_relevancy': 0.8982, 'context_precision': 0.9833, 'context_recall': 0.9750, 'answer_similarity': 0.9634, 'answer_correctness': 0.7417}


In [54]:
avg_result_search_query_gpt4 = dictionary(result_search_query_gpt4)

The average score is: 0.9254145421692223
